</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode
from methods.llm import get_groq_response

# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None, get_response=get_groq_response)
node.get_test_cases(2)
node.evolve("i1", replace=True)

input_dict = {"name": "Dilireba"}
output_dict = node(input_dict) 

Could not load vllm class, check CUDA support and GPU RAM size


Generating test cases: 5case [00:01,  2.60case/s]             


--- Generated 5 test cases
Error in parsing LLM response: 
 JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1) 
Response: 
 To find the birthdate of Leonardo DiCaprio and calculate his age, we'll use a combination of the `webbrowser` and `datetime` libraries in Python to retrieve his birthdate from Wikipedia.

```python
import webbrowser
import datetime

def get_celebrity_birthdate():
    # Search for Leonardo DiCaprio's birthdate on Wikipedia
    query = f"https://en.wikipedia.org/wiki/Leonardo_DiCaprio"
    webbrowser.open(query, new=2)  # Open the link in a new tab/window
    
    # Unfortunately, there's no way to scrape a website directly using a library
    # without getting into hairy web scraping laws. Here's a hypothetical example
    # for demonstration purposes. Please replace this with the actual scraping
    # if needed.
    birthdate_str = "November 11, 1974"
    
    # Process the birthdate string
    birthda

#### Direct Loading Node

In [1]:
from methods.eoh_evolution import EvolNode 

node = EvolNode.load("get_celeb_age")
input_dict = {"name": "Dilireba"}
node(input_dict)

Could not load vllm class, check CUDA support and GPU RAM size


{'age': 31}

#### Alignment-Check for Evaluation

In [2]:
from methods.llm import get_groq_response 

# Evaluation base on alignment check
node.get_response = get_groq_response 
structure_fitness, function_fitness, issue_summary = node._evaluate_fitness(max_tries=1, num_runs=2)

Evaluating fitness: 100%|██████████| 4/4 [00:13<00:00,  3.39s/it]


In [3]:
print(issue_summary)

--- Compiled 0 out of 4 test cases
--- Passed 0 out of 4 test cases
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.



In [5]:
# how to put a node into another nodes? 
# Ideally I would then ask for the node (which likely fails on many of the test-cases) to come-up with a code, where it could call on some 'pseudo-node'
# These pseudo-nodes are used, very similar to a pseudo-code 




</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>

</div>
<div align="center">
  <img src="img/Planning-node.png" width="400" alt="Planning Node Task Decomposition">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [2]:
from methods.llm import get_claude_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan,extract_python_code, extract_json_from_text

# Step 0. Initialize MetaPlanning Prompt
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])

# Step 1: Generate Pseudo-Code for SubTask Decomposition
prompt = mp._get_pseudo_code_prompt() # Pseudo-Code Prompt (Non-implemented functional)
response = get_claude_response(prompt) # Use Strong LLM to build up pseudo-code
code = extract_python_code(response) # Extract Python Code from response 

# Step 2: Generate Planning DAG: Multiple Nodes 
graph_prompt = mp._get_plan_graph_prompt(code) 
plan_response = get_claude_response(graph_prompt)
plan_dict = extract_json_from_text(plan_response)

visualize_plan_dict(plan_dict)

# Step 3: Spawn Multiple Nodes and Evolve them

In [24]:
# visualize_plan_dict(plan_dict)

plan_dict # Ignore the mode for now, let's use both mode and pick the best performing ones ... 

plan_dict["nodes"]

# Basic Idea: 
# - Build node libraries is the better way to describe this 



[{'task': 'Search for celebrity in database',
  'name': 'search_celebrity_database',
  'inputs': ['name'],
  'input_types': ['str'],
  'outputs': ['celeb_info'],
  'output_types': ['dict'],
  'target': 'Retrieve celebrity information',
  'mode': 'CODE'},
 {'task': 'Extract birth date from celebrity info',
  'name': 'extract_birth_date',
  'inputs': ['celeb_info'],
  'input_types': ['dict'],
  'outputs': ['birth_date'],
  'output_types': ['date'],
  'target': "Get celebrity's birth date",
  'mode': 'CODE'},
 {'task': 'Get current date',
  'name': 'get_current_date',
  'inputs': [],
  'input_types': [],
  'outputs': ['current_date'],
  'output_types': ['date'],
  'target': 'Obtain current date for age calculation',
  'mode': 'CODE'},
 {'task': 'Calculate age based on birth date and current date',
  'name': 'calculate_age',
  'inputs': ['birth_date', 'current_date'],
  'input_types': ['date', 'date'],
  'outputs': ['age'],
  'output_types': ['int'],
  'target': "Determine celebrity's curr